https://katex.org/docs/supported.html

# OLP: Online Linear Programming



### Pyomo_0

First version of OLP using Pyomo.  
This example relates to the ``pyomo_0`` sheet in this workbook om OneDrive: [TGC_LP](https://1drv.ms/x/s!AiogHeTeve1hj5NFFlBANUYBfoFAcA?e=Kj0bdQ)


$$
\begin{array}{lllll}
\text{min} & \displaystyle\sum_{j=1}^{n=5} \left( e_{j}  -  \displaystyle\sum_{i=1}^{m=1} x_{ij} \right) & \\\\
\text{s.t.} & \sum_{i=1}^{m} x_{ij} \leq e_{j} & \forall i = 1, \ldots, m \\\\
& x_{ij} \geq 0 & \forall j = 1, \ldots, n 
\end{array} 
$$



$$
\begin{array}{ll}
\text{i =  EVSE number}  \\
\text{j =  time slot}
\end{array} 
$$



#### Excel-model: results

<img src="./images/pyomo_o.png" width="1000">

<br>





<div style="border:1px solid blue; border-radius: 5px; background-color: #DDEDF9; color: black; padding: 0px; width: 50%">
    <div style="background-color: darkblue; color: white; padding: 10px; border-radius: 5px 5px 0 0; width: 97%">
        <strong>Note:</strong>
    </div>
    <BR>
    >  The outcomes of the excel-model are the same as the pyomo-model !!!
    <BR>
    <BR>
</div>

### Pyomo-model: results

In [5]:
# Tetris Game Charger

# from __future__ import division
from pyomo.environ import *

model = AbstractModel()

model.I = Set()  # set of EVSEs
model.J = Set()  # set of time periods for a certain horizon (h=5)

model.ex = Param(model.J)  # enexis max Power output for each time period (constant)
model.ev = Param(model.I, model.J)  # EV max Power input for each time period
model.se = Param(model.I, model.J)  # EVSE max Power output for each time period

# the next line declares a variable indexed by the set J
model.x = Var(model.I, model.J, domain=NonNegativeReals)


def obj_expression(model):
    # return the expression for the objective
    return sum(model.ex[j] - sum(model.x[i, j] for i in model.I) for j in model.J)


model.OBJ = Objective(rule=obj_expression)


def demand_constraint_rule(model, j):
    # return the expression for the constraint for j
    return sum(model.x[i, j] for i in model.I) <= model.ex[j]


def ev_mpi_constraint_rule(model, i, j):
    # return the expression for the constraint for j
    return model.x[i, j] <= model.ev[i, j]


def se_mpo_constraint_rule(model, i, j):
    # return the expression for the constraint for j
    return model.x[i, j] <= model.se[i, j]


# the next line creates one constraint for each member of the set model.J
model.PwrDmdConstraint = Constraint(model.J, rule=demand_constraint_rule)
model.EV_MPIConstraint = Constraint(model.I, model.J, rule=ev_mpi_constraint_rule)
model.SE_MPOConstraint = Constraint(model.I, model.J, rule=se_mpo_constraint_rule)

# --------------------------------------------------------------------------
# data section
m = 1
n = 5

data = {
    None: {
        "I": {None: list(range(1, m + 1))},
        "J": {None: list(range(1, n + 1))},
        "ev": {
            (1, 1): 7.36,
            (1, 2): 7.36,
            (1, 3): 5.00,
            (1, 4): 3.68,
            (1, 5): 0.0,
        },  # EV   max power input  per time period
        "se": {
            (1, 1): 5.00,
            (1, 2): 7.36,
            (1, 3): 7.36,
            (1, 4): 7.36,
            (1, 5): 7.36,
        },  # EVSE max power output per time period
        "ex": {1: 14.72, 2: 14.72, 3: 14.72, 4: 14.72, 5: 14.72},
    }
}

# --------------------------------------------------------------------------
# create a model instance and optimize

tgc = model.create_instance(data)

opt = pyomo.environ.SolverFactory("cplex")  # or 'mosek' or 'glpk'

opt.solve(tgc)

# --------------------------------------------------------------------------
# display solution
tgc.pprint()
[tgc.x[1, x].value for x in range(1, 6)]
print("\nObjective = ", value(tgc.OBJ))


7 Set Declarations
    EV_MPIConstraint_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    I*J :    5 : {(1, 1), (1, 2), (1, 3), (1, 4), (1, 5)}
    I : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    1 :    {1,}
    J : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    SE_MPOConstraint_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    I*J :    5 : {(1, 1), (1, 2), (1, 3), (1, 4), (1, 5)}
    ev_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    I*J :    5 : {(1, 1), (1, 2), (1, 3), (1, 4), (1, 5)}
    se_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    I*J :    5 : {(1, 1